In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import scanpy as sc

import seaborn as sns
from matplotlib import pyplot as plt

/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
print(ad.shape)

(67988, 33538)


In [3]:
np.unique(ad.obs.celltype)

array(['Endothelial', 'Epithelial', 'Fibroblast', 'Lymphocyte', 'Myeloid'],
      dtype=object)

# Derive gene signatures

### Repeat the procedure for each set of celltype

celltype groups:

    - Epithelial
    - Fibroblast
    - Lymphocyte, Myeloid


In [4]:
ad = sc.read_h5ad("../data/MIBC_snSeq.h5ad")
ad = ad[ad.obs.celltype.isin(['Epithelial'])]

rm_genes = list(ad.var_names[ad.var_names.str.contains('\.')])
rm_genes += list(ad.var_names[ad.var_names.str.startswith('MT-')])

ad.raw = ad
sc.pp.normalize_total(ad, target_sum=10000)
sc.pp.log1p(ad)

ad = ad[:, ~ad.var_names.isin(rm_genes)]

/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
Trying to set attribute `.var` of view, copying.


(61019, 23383) 7191


In [6]:
sc.tl.rank_genes_groups(ad, groupby='subtype', method='wilcoxon', n_genes=200)

/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1192: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])
Trying to set attribute `.uns` of view, copying.


In [7]:
cols = np.unique(ad.obs.subtype)
all_genes = []
for c in cols:
    if c == 'Endothelial':
        continue
    df = sc.get.rank_genes_groups_df(ad, c)
    all_genes += list(df.names)
    
all_markers = pd.DataFrame(index=cols, columns=np.unique(all_genes))
for c in cols:
    if c == 'Endothelial':
        continue
    df = sc.get.rank_genes_groups_df(ad, c)
    df = df.set_index('names')
    all_markers.loc[c, list(df.index)] = df.scores

In [9]:
print(all_markers.shape)
signatures = pd.DataFrame({'subtype': all_markers.index.values[np.argmax(all_markers.fillna(0).values, axis=0)]}, index=all_markers.columns)
signatures.index.name = 'gene'
signatures = signatures.sort_values('subtype')

# signatures.to_csv()

(6, 910)
